## Lesson 2: RAG Triad of metrics

#### This code is a reproduction of the Lesson 2 Building and Evaluating Advanced RAG Course by Deep Learning AI

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [4]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()



🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [5]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [7]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [8]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [9]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'You create your AI portfolio by showcasing a progression of projects that demonstrate your skills.'

## Feedback functions

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [12]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [13]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [15]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [16]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [17]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [18]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [19]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [20]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [21]:
eval_questions.append("How can I be successful in AI?")

In [22]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [23]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_4bbe2a1adb521953ed854bfbd3a61cce,"""What are the keys to building a career in AI?""","""The keys to building a career in AI involve l...",-,"{""record_id"": ""record_hash_4bbe2a1adb521953ed8...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-22T17:46:20.596959"", ""...",2024-08-22T17:46:23.896248,1.0,0.5,1.000000,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'question': 'What are the keys to b...,[{'args': {'source': 'Chapter 7: A Simple Fram...,3,509,0.000779
1,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_62f5f106e629779d9f979748546b2731,"""How can teamwork contribute to success in AI?""","""Teammates play a crucial role in the success ...",-,"{""record_id"": ""record_hash_62f5f106e629779d9f9...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-22T17:46:24.060443"", ""...",2024-08-22T17:46:26.854570,0.9,0.8,0.566667,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'question': 'How can teamwork contr...,[{'args': {'source': 'To get a project starte...,2,637,0.000996
2,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6795dcef0d866e750ac67eb2abf444c3,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide...",-,"{""record_id"": ""record_hash_6795dcef0d866e750ac...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-22T17:46:26.956611"", ""...",2024-08-22T17:46:29.624309,1.0,0.6,0.000000,[{'args': {'prompt': 'What is the importance o...,[{'args': {'question': 'What is the importance...,[{'args': {'source': 'What is the hiring proce...,2,538,0.000859
3,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e5a3b3320e4bddcac4de1ffc44a300c2,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_e5a3b3320e4bddcac4d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-22T17:46:29.715416"", ""...",2024-08-22T17:46:32.696773,1.0,0.6,0.433333,[{'args': {'prompt': 'What are some good habit...,[{'args': {'question': 'What are some good hab...,"[{'args': {'source': '(For example, many arti...",2,434,0.000669
4,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_8ac7dc263d2ba79165bca02c19221e23,"""How can altruism be beneficial in building a ...","""Helping others and lifting them up during one...",-,"{""record_id"": ""record_hash_8ac7dc263d2ba79165b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-22T17:46:32.792916"", ""...",2024-08-22T17:46:34.098883,1.0,NaN,1.000000,[{'args': {'prompt': 'How can altruism be bene...,NaN,[{'args': {'source': 'Many successful people d...,1,482,0.000734


In [25]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Answer Relevance,Groundedness,Context Relevance
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI involve learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.""",1.0,1.000000,0.5
1,"""How can teamwork contribute to success in AI?""","""Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on building AI for the benefit of all can positively influence one's own work ethic and outcomes. The ability to work effectively in a team, leveraging each member's strengths and insights, can lead to improved project outcomes and personal growth as a leader in AI projects.""",0.9,0.566667,0.8
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current trends. Networking also allows for the exchange of ideas, collaboration on projects, and access to job opportunities that may not be advertised publicly. Overall, networking in AI can help individuals build a strong support system, stay informed about developments in the field, and enhance their career prospects.""",1.0,0.000000,0.6
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while maintaining their health.""",1.0,0.433333,0.6
4,"""How can altruism be beneficial in building a career?""","""Helping others and lifting them up during one's own career journey can lead to better outcomes for oneself.""",1.0,1.000000,NaN
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a phenomenon where individuals doubt their accomplishments and have a persistent fear of being exposed as a fraud. In the context of AI, newcomers to the field may experience imposter syndrome, feeling like they do not truly belong in the AI community despite their success. This can be a significant challenge for individuals in AI, but it is important to address and overcome in order to continue growing and contributing to the field.""",1.0,0.550000,NaN
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",1.0,1.000000,NaN
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",1.0,0.800000,NaN
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with rapidly evolving technologies, the iterative nature of AI projects leading to uncertainties in project management, the need to collaborate with stakeholders who may lack expertise in AI, difficulties in finding suitable projects and estimating timelines and returns on investment, as well as the challenge of tuning neural network hyperparameters accurately.""",1.0,NaN,NaN
9,"""Is it normal to find parts of AI challenging?""","""It is normal to find parts of AI challenging.""",1.0,NaN,NaN


In [26]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
App_1,0.981818,0.736364,0.627273,1.636364,0.000841


In [27]:
tru.run_dashboard()

Starting dashboard ...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.27:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>